# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use exchange-specific
characterization factors in the characterization matrix of `bw2calc`, combining the use of exchange names and locations.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import EdgeLCIA, get_available_methods
import bw2data

One can simply build its own LCIA file.
In this example, we will consider the use of parameters (saved under `lcia_example_3.json`).
Here, we can see that the evaluation of the CF for methane and dinitrogen monoxide depends on the value of the parameter `co2ppm`. It is a simple approximation of the GWP100 factor of these gases as a function of the atmospheric CO₂ concentration.

In [2]:
[
  {
    "supplier": {
      "name": "Carbon dioxide",
      "operator": "startswith",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": "1.0"
  },
  {
    "supplier": {
      "name": "Methane, fossil",
      "operator": "contains",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": "28 * (1 + 0.001 * (co2ppm - 410))"
  },
  {
    "supplier": {
      "name": "Dinitrogen monoxide",
      "operator": "equals",
      "matrix": "biosphere"
    },
    "consumer": {
      "matrix": "technosphere",
      "type": "process"
    },
    "value": "265 * (1 + 0.0005 * (co2ppm - 410))"
  }
]

[{'supplier': {'name': 'Carbon dioxide',
   'operator': 'startswith',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': '1.0'},
 {'supplier': {'name': 'Methane, fossil',
   'operator': 'contains',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': '28 * (1 + 0.001 * (co2ppm - 410))'},
 {'supplier': {'name': 'Dinitrogen monoxide',
   'operator': 'equals',
   'matrix': 'biosphere'},
  'consumer': {'matrix': 'technosphere', 'type': 'process'},
  'value': '265 * (1 + 0.0005 * (co2ppm - 410))'}]

We can instantiate the EdgeLCIA() class as usual, except that we need to pass the parameters as a dictionary.
Then we proceed to the mapping steps. Finally, we iterate over the scenarios and evaluate the CFs.

In [3]:

import bw2data
from edges import EdgeLCIA

# Select an activity from the LCA database
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
print(act)

# Define scenario parameters (e.g., atmospheric CO₂ concentration and time horizon)
params = {"co2ppm": [410, 450, 500], "h": 100}

# Define an LCIA method (symbolic CF expressions stored in JSON)
method = ('GWP', 'scenario-dependent', '100 years')

# Initialize LCIA
lcia = EdgeLCIA(
   demand={act: 1},
   filepath="lcia_example_3.json",
   parameters=params
)

# Perform inventory calculations (once)
lcia.lci()

# Map exchanges to CF entries (once)
lcia.map_exchanges()

# Optionally, resolve geographic overlaps and disaggregations (once)
lcia.map_aggregate_locations()
lcia.map_dynamic_locations()
lcia.map_remaining_locations_to_global()

# Run scenarios efficiently
results = []
for idx in range(lcia.scenario_length):
    lcia.evaluate_cfs(idx)
    lcia.lcia()
    df = lcia.generate_cf_table()

    scenario_result = {
        "scenario": idx,
        "co2ppm": params["co2ppm"][idx],
        "score": lcia.score,
        "CF_table": df
    }
    results.append(scenario_result)

    print(f"Scenario {idx+1} (CO₂ {params['co2ppm'][idx]} ppm): Impact = {lcia.score}")

/opt/homebrew/Caskroom/miniforge/base/envs/bw2/lib/python3.11/site-packages/bw2data/backends/peewee/database.py:127: UserWarning: This database is empty
  warnings.warn("This database is empty")


None


ValueError: Invalid demand dictionary